In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
LEARNING_RATE = 1e-4
EPOCHS = 500
PATIENCE = 20
BATCH_SIZE = 32

DIRECTORY = "data"
CATEGORIES = [
  "K", "O", "nothing"
  # "A", "B", "C", "D", "E", "F", "G", "H", "I", "J",
  # "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T",
  # "U", "V", "W", "X", "Y", "Z", "del", "nothing", "space"
]

In [ ]:
test_data = []
test_labels = []

for category in CATEGORIES:
  path = os.path.join(DIRECTORY, "temp_train", category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    test_data.append(image)
    test_labels.append(category)

In [ ]:
print(test_data[0])

In [ ]:
# One-hot encoding on the labels
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

In [ ]:
test_data = np.array(test_data, dtype="float32")
test_labels = np.array(test_labels)

In [ ]:
print(test_labels[0])

In [ ]:
mobileNet = MobileNetV2(
  weights="imagenet", 
  include_top=False, 
  input_tensor=Input(shape=(224, 224, 3))
)

headModel = mobileNet.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CATEGORIES), activation="softmax")(headModel)

model = Model(inputs=mobileNet.input, outputs=headModel)

In [ ]:
# We don't want new trainings to affect the MobileNetV2 weights
# Why? Because Indian said it :D
for layer in mobileNet.layers:
  layer.trainable = False

In [ ]:
optimizer = Adam(learning_rate=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
model.compile(
  loss="categorical_crossentropy", 
  optimizer=optimizer, 
  metrics=["accuracy"]
)

early_stopping = EarlyStopping(
  monitor='val_loss', 
  patience=PATIENCE, 
  restore_best_weights=True
)

In [ ]:
history = model.fit(
  test_data, test_labels, 
  batch_size=BATCH_SIZE, 
  validation_split=0.1,
  epochs=EPOCHS,
  verbose=1,
  callbacks=[early_stopping]
)

In [ ]:
# TODO: load test data and test model

# Save the model
model.save("models/sign_language_model.keras")

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), history.history["val_accuracy"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), history.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")